In [1]:
import pandas as pd

In [2]:
datasets = pd.read_csv('Boston_Housing.csv')
datasets

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


In [3]:
datasets.shape
datasets[:5]

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [4]:
datasets = pd.concat([datasets["MEDV"],
                     datasets.drop(['MEDV'], axis = 1)],
                     axis= 1  )

In [5]:
# datasets

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
training_datasets, validation_datasets = train_test_split(datasets, test_size =0.1)

In [8]:
# training_datasets

In [9]:
# TO CREAT A NEW 90% TRAINING SPLITED FILE NAMELY TRAINIING_DATASETS  
training_datasets.to_csv('training_datasets.csv', 
                        index = False, header=False)

In [10]:
# import os
# os.listdir()

In [7]:
# TO CREAT A NEW 10% VALIDATION SPLITED FILE NAMELY VALIDATION _DATASETS  
validation_datasets.to_csv('validation_datasets.csv', 
                        index = False, header=False)

In [8]:
print(training_datasets.shape)
validation_datasets.shape

(455, 14)


(51, 14)

In [9]:
import sagemaker

In [10]:
sess = sagemaker.Session()
bucket = sess.default_bucket()

In [15]:
bucket

'sagemaker-us-east-2-182630706329'

In [11]:
prefix = 'boston-housing'

In [31]:
training_data_path = sess.upload_data( path='training_datasets.csv',
 key_prefix=prefix + '/input/training')

In [32]:
training_data_path

's3://sagemaker-us-east-2-182630706329/boston-housing/input/training/training_datasets.csv'

In [33]:
validation_data_path = sess.upload_data(
path='validation_datasets.csv',
key_prefix=prefix + '/input/validation')

In [13]:
validation_data_path

's3://sagemaker-us-east-2-182630706329/boston-housing/input/validation/validation_datasets.csv'

In [17]:
import boto3
from sagemaker import image_uris

In [18]:
region = boto3.Session().region_name
container = image_uris.retrieve('linear-learner', region)

In [19]:
print(region)
container

us-east-2


'404615174143.dkr.ecr.us-east-2.amazonaws.com/linear-learner:1'

In [20]:
from sagemaker.estimator import Estimator

In [23]:
import boto3
region = boto3.Session().region_name

def resolve_sm_role():
    client = boto3.client('iam', region_name=region)
    response_roles = client.list_roles(
        PathPrefix='/',
        # Marker='string',
        MaxItems=999
    )
    for role in response_roles['Roles']:
        if role['RoleName'].startswith('AmazonSageMaker-ExecutionRole-'):
            #print('Resolved SageMaker IAM Role to: ' + str(role))
            return role['Arn']
    raise Exception('Could not resolve what should be the SageMaker role to be used')

role = resolve_sm_role()
print(role)

arn:aws:iam::182630706329:role/service-role/AmazonSageMaker-ExecutionRole-20210304T231049


In [57]:
ll_estimator = Estimator(
 container,
    role = 'arn:aws:iam::182630706329:role/service-role/AmazonSageMaker-ExecutionRole-20210304T231049',
#  role=sagemaker.get_execution_role(),
 instance_count=1,
 instance_type='ml.m5.large',
 output_path='s3://{}/{}/output'.format(bucket,
 prefix))

In [58]:
ll_estimator.set_hyperparameters(
 predictor_type='regressor',
 mini_batch_size=32)

In [59]:
training_data_channel = sagemaker.TrainingInput(
 s3_data=training_data_path, content_type='text/csv')

In [60]:
validation_data_channel = sagemaker.TrainingInput(
 s3_data=validation_data_path, content_type='text/csv')

In [ ]:
ll_estimator.fit({'train': training_data_channel,
 'validation': validation_data_channel})

2021-03-27 13:24:33 Starting - Starting the training job...
2021-03-27 13:24:55 Starting - Launching requested ML instancesProfilerReport-1616851481: InProgress
...
2021-03-27 13:25:39 Starting - Preparing the instances for training......
2021-03-27 13:26:56 Downloading - Downloading input data
2021-03-27 13:26:56 Training - Downloading the training image

In [61]:
from time import strftime, gmtime

In [62]:
timestamp = strftime('%d-%H-%M-%S', gmtime())
endpoint_name = 'linear-learner-demo-'+timestamp
print(endpoint_name)

linear-learner-demo-27-13-24-26


In [ ]:
ll_predictor = ll_estimator.deploy(
 endpoint_name=endpoint_name,
 initial_instance_count=1,
 instance_type='ml.t2.medium')

In [ ]:
# ll_predictor.content_type = 'text/csv'
ll_predictor.serializer   = sagemaker.serializers.CSVSerializer()
ll_predictor.deserializer = sagemaker.deserializers.CSVDeserializer()

In [ ]:
test_sample = '0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,4.98'

In [ ]:
response = ll_predictor.predict(test_sample,content_type ='text/csv')

In [ ]:
print(response)

In [ ]:
test_samples = ['0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,4.98',
                '0.02731,0.00,7.070,0,0.4690,6.4210,78.90,4.9671,2,242.0,
17.80,9.14']

In [ ]:
response = ll_predictor.predict(test_samples)
print(response)

In [ ]:
ll_predictor.delete_endpoint()

In [ ]:
# train_data = 's3://{}/{}/{}'.format(bucket, prefix, 'train/examples') #making train_data

# #making test_data
# test_key = "{}/test/examples".format(prefix)
# s3.Bucket(bucket).download_file(test_key, 'test_data')

# #preparing train channels for training the data
# train_channel = sagemaker.session.s3_input(train_data, content_type='text/csv')

# #training the data
# linear.fit(inputs=train_channel,  logs=True)

# #creating the endpoint
# linear_predictor = linear.deploy(initial_instance_count=1,
#                              instance_type='ml.m5.large')

In [ ]:
# l = []
# with open('test_samples', 'r') as f:
    
#     for j in range(0,56962):
#         single_test = f.readline()
#         result = linear_`predictor.predict(single_test)
#         l.append(result)
#         if j%10000 ==0 :
#             print(j)
#     print(l[0:10])

#     #getting the results on train_data
#     #THE CODE BELOW IS THROWING THE ABOVE MENTIONED ERROR
#     q =[]
#     with open('train_samples', 'r') as f:
        
#         for j in range(0,56962):
#             single_test = f.readline()
#             result = linear_predictor.predict(single_test)
#             q.append(result)
#             if j%10000 ==0 :
#                 print(j)
#         print(q[0:10])

# Same work with XG BOOST Algo

In [21]:
import boto3
region = boto3.Session().region_name
container = image_uris.retrieve('xgboost', region,
 version='latest')

In [24]:
xgb_estimator = Estimator(
 container,
 role=role,
 instance_count=1,
 instance_type='ml.m5.large',
 output_path='s3://{}/{}/output'.format(bucket,
 prefix))

In [25]:
xgb_estimator.set_hyperparameters(
 objective='reg:linear',
 num_round=200,
 early_stopping_rounds=10)

In [34]:
training_data_channel = sagemaker.TrainingInput(
 s3_data=training_data_path,
 content_type='text/csv')

In [35]:
validation_data_channel = sagemaker.TrainingInput(
 s3_data=validation_data_path,
 content_type='text/csv')

In [36]:
xgb_estimator.fit({'train': training_data_channel,
 'validation': validation_data_channel})

2021-03-27 12:14:35 Starting - Starting the training job...
2021-03-27 12:15:09 Starting - Launching requested ML instancesProfilerReport-1616847284: InProgress
......
2021-03-27 12:16:12 Starting - Preparing the instances for training......
2021-03-27 12:17:32 Downloading - Downloading input data
2021-03-27 12:17:32 Training - Downloading the training image.Arguments: train
[2021-03-27:12:17:39:INFO] Running standalone xgboost training.
[2021-03-27:12:17:39:INFO] File size need to be processed in the node: 0.03mb. Available memory size in the node: 114.16mb
[2021-03-27:12:17:39:INFO] Determined delimiter of CSV input is ','
[12:17:39] S3DistributionType set as FullyReplicated
[12:17:39] 455x13 matrix with 5915 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2021-03-27:12:17:39:INFO] Determined delimiter of CSV input is ','
[12:17:39] S3DistributionType set as FullyReplicated
[12:17:39] 51x13 matrix with 663 entries loaded from /opt/ml/input/data/val


2021-03-27 12:18:12 Uploading - Uploading generated training model
2021-03-27 12:18:12 Completed - Training job completed
Training seconds: 51
Billable seconds: 51


In [50]:
from time import strftime, gmtime
timestamp = strftime('%d-%H-%M-%S', gmtime())
endpoint_name = 'xgb-demo'+'-'+timestamp
xgb_predictor = xgb_estimator.deploy(endpoint_name=endpoint_name, initial_instance_count=1, instance_type='ml.t2.medium')

ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateEndpoint operation: The account-level service limit 'Number of instances across active endpoints' is 2 Instances, with current utilization of 2 Instances and a request delta of 1 Instances. Please contact AWS support to request an increase for this limit.

In [45]:
test_sample = '0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,4.98'

In [48]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()
xgb_predictor.deserializer = sagemaker.deserializers.CSVDeserializer()

NameError: name 'xgb_predictor' is not defined